In [1]:
%matplotlib inline

from time import process_time
import numpy as np
import pandas as pd
from mdptoolbox import mdp
from itertools import product

In [2]:
# Set up Gridworld
grid_size = (3, 4)
blocked_cell = (1, 1)
baseline_reward = -0.02
absorbing_cells = {(0, 3): 1, (1, 3): -1}
actions = ['L', 'U', 'R', 'D']
num_actions = len(actions)
probs = [.1, .8, .1, 0]

In [3]:
to_1d = lambda x: np.ravel_multi_index(x, grid_size)
to_2d = lambda x: np.unravel_index(x, grid_size)

In [4]:
num_states = np.product(grid_size)
cells = list(np.ndindex(grid_size))
states = list(range(len(cells)))
cell_state = dict(zip(cells, states))
state_cell= dict(zip(states, cells))
absorbing_states = {to_1d(s):r for s, r in absorbing_cells.items()}
blocked_state = to_1d(blocked_cell)

In [5]:
state_rewards = np.full(num_states, baseline_reward)
state_rewards[blocked_state] = 0
for state, reward in absorbing_states.items():
    state_rewards[state] = reward

action_outcomes = {}
for i, action in enumerate(actions):
    probs_ = dict(zip([actions[j % 4] for j in range(i, num_actions + i)], probs))
    action_outcomes[actions[(i + 1) % 4]] = probs_

action_outcomes

{'U': {'L': 0.1, 'U': 0.8, 'R': 0.1, 'D': 0},
 'R': {'U': 0.1, 'R': 0.8, 'D': 0.1, 'L': 0},
 'D': {'R': 0.1, 'D': 0.8, 'L': 0.1, 'U': 0},
 'L': {'D': 0.1, 'L': 0.8, 'U': 0.1, 'R': 0}}

In [6]:
# Transition Matrix
def get_new_cell(state, move):
    cell = to_2d(state)
    if actions[move] == 'U':
        return cell[0] - 1, cell[1]
    elif actions[move] == 'D':
        return cell[0] + 1, cell[1]
    elif actions[move] == 'R':
        return cell[0], cell[1] + 1
    elif actions[move] == 'L':
        return cell[0], cell[1] - 1

state_rewards

array([-0.02, -0.02, -0.02,  1.  , -0.02,  0.  , -0.02, -1.  , -0.02,
       -0.02, -0.02, -0.02])

In [7]:
def update_transitions_and_rewards(state, action, outcome):
    if state in absorbing_states.keys() or state == blocked_state:
        transitions[action, state, state] = 1
    else:
        new_cell = get_new_cell(state, outcome)
        p = action_outcomes[actions[action]][actions[outcome]]
        if new_cell not in cells or new_cell == blocked_cell:
            transitions[action, state, state] += p
            rewards[action, state, state] = baseline_reward
        else:
            new_state= to_1d(new_cell)
            transitions[action, state, new_state] = p
            rewards[action, state, new_state] = state_rewards[new_state]

In [8]:
rewards = np.zeros(shape=(num_actions, num_states, num_states))
transitions = np.zeros((num_actions, num_states, num_states))
actions_ = list(range(num_actions))
for action, outcome, state in product(actions_, actions_, states):
    update_transitions_and_rewards(state, action, outcome)

rewards.shape, transitions.shape

((4, 12, 12), (4, 12, 12))

In [9]:
# PyMDPToolbox

# Value Iteration
gamma = .99
epsilon = 1e-5

vi = mdp.ValueIteration(transitions=transitions, reward=rewards, discount=gamma, epsilon=epsilon)

vi.run()
print(f'# Iterations: {vi.iter:,d} | Time: {vi.time:.4f}')

# Iterations: 31 | Time: 0.0010


In [10]:
policy = np.asarray([actions[i] for i in vi.policy])
pd.DataFrame(policy.reshape(grid_size))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,L,L,L


In [11]:
value = np.asarray(vi.V).reshape(grid_size)
pd.DataFrame(value)

,0,1,2,3
0,0.884143,0.925054,0.961986,0.000000
1,0.848181,0.000000,0.714643,0.000000
2,0.808345,0.773328,0.736099,0.516083


In [12]:
# Policy Iteration
pi = mdp.PolicyIteration(transitions=transitions, reward=rewards, discount=gamma, max_iter=1000)

pi.run()
print(f'# Iterations: {pi.iter:,d} | Time: {pi.time:.4f}')

# Iterations: 7 | Time: 0.0419


In [13]:
policy = np.asarray([actions[i] for i in pi.policy])
pd.DataFrame(policy.reshape(grid_size))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,L,L,L


In [14]:
value = np.asarray(pi.V).reshape(grid_size)
pd.DataFrame(value)

,0,1,2,3
0,0.884143,0.925054,0.961986,1.594721e-16
1,0.848181,0.000000,0.714643,-0.000000e+00
2,0.808345,0.773328,0.736099,5.160828e-01


In [15]:
# Value Iteration
skip_states = list(absorbing_states.keys())+[blocked_state]
states_to_update = [s for s in states if s not in skip_states]

V = np.random.rand(num_states)
V[skip_states] = 0
gamma = .99
epsilon = 1e-5

In [16]:
iterations = 0
start = process_time()
converged = False
while not converged:
    V_ = np.copy(V)
    for state in states_to_update:
        q_sa = np.sum(transitions[:, state] * (rewards[:, state] + gamma* V), axis=1)
        V[state] = np.max(q_sa)
    if np.sum(np.fabs(V - V_)) < epsilon:
        converged = True

    iterations += 1
    if iterations % 1000 == 0:
        print(np.sum(np.fabs(V - V_)))

print(f'# Iterations {iterations} | Time {process_time() - start:.4f}')
print(pd.DataFrame(V.reshape(grid_size)))

# Iterations 29 | Time 0.0000
          0         1         2         3
0  0.884143  0.925054  0.961986  0.000000
1  0.848181  0.000000  0.714643  0.000000
2  0.808345  0.773328  0.736099  0.516083


In [17]:
np.allclose(V.reshape(grid_size), np.asarray(vi.V).reshape(grid_size))

True

In [18]:
# Optimal Policy
for state, reward in absorbing_states.items():
    V[state] = reward

policy = np.argmax(np.sum(transitions * V, 2),0)
policy

array([2, 2, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0], dtype=int64)

In [19]:
pd.DataFrame(policy.reshape(grid_size)).replace(dict(enumerate(actions)))

,0,1,2,3
0,R,R,R,L
1,U,L,L,L
2,U,L,L,L


In [20]:
# Policy Iteration
def policy_improvement(value, transitions):
    for state, reward in absorbing_states.items():
        value[state] = reward
    return np.argmax(np.sum(transitions * value, 2),0)

V = np.random.rand(num_states)
V[skip_states] = 0
pi = np.random.choice(list(range(num_actions)), size=num_states)

In [21]:
iterations = 0
start = process_time()
converged = False
while not converged:
    pi_ = np.copy(pi)
    for state in states_to_update:
        action = policy[state]
        V[state] = np.dot(transitions[action, state], (rewards[action, state] + gamma* V))
        pi = policy_improvement(V.copy(), transitions)
    if np.array_equal(pi_, pi):
        converged = True
    iterations += 1

print(f'# Iterations {iterations} | Time {process_time() - start:.4f}')

# Iterations 3 | Time 0.0000


In [22]:
pd.DataFrame(pi.reshape(grid_size)).replace(dict(enumerate(actions)))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,L,U,L


In [23]:
pd.DataFrame(V.reshape(grid_size))

,0,1,2,3
0,0.814431,0.919626,0.955838,0.000000
1,0.731303,0.000000,0.634186,0.000000
2,0.647915,0.577527,0.541438,0.333944
